<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/cDNA-pretraining/blob/main/experiments/kmer_tokenization/SentencePieceTokenizer_for_DNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
!pip install -qq transformers datasets --quiet

In [5]:
# #CONFIG
hf_name = 'Vlasta'
vocab_limit = 30000

In [6]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [7]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Split
from tokenizers.processors import BertProcessing
from tokenizers import Regex
from tokenizers import SentencePieceBPETokenizer, SentencePieceUnigramTokenizer

In [25]:
ds = load_dataset("simecek/Human_DNA_v0")
train_portion = ds['train']['Seq']

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
len(train_portion)

263659

# Google implementation training

## Creating corpus

In [9]:
!mkdir train_texts
!mkdir big_train_text

mkdir: cannot create directory ‘train_texts’: File exists
mkdir: cannot create directory ‘big_train_text’: File exists


In [8]:
file_path='big_train_text/corpus.txt'


In [26]:
import math
tf = open(file_path, 'w')
# chunk_size = 1000
for i,el in enumerate(train_portion): #Limit dataset option
  # for j in range(math.ceil(len(el)/chunk_size)):
    # tf.write(el[j*chunk_size: (j+1)*chunk_size]+'\n')

  tf.write(el+'\n')

tf.close()

## Training

In [6]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import sentencepiece as spm
max_sen_len = 10001 #dataset max is 10000+1 #default 4192
max_token_len = 100 #Default 16
#https://github.com/google/sentencepiece/blob/master/doc/options.md
hard_vocab_limit = True #default=True

input_sentence_size=15000 #Limits num of senteces sampled

vocab_limit-=1 #MASK token is added later
spm.SentencePieceTrainer.train(
  f"--input={file_path} "\
  f"--max_sentence_length={max_sen_len} "\
  f"--max_sentencepiece_length={max_token_len} "\
  f"--model_prefix=myBPE "\
  f"--vocab_size={vocab_limit} "\
  f"--hard_vocab_limit={hard_vocab_limit} "\
  f"--input_sentence_size={input_sentence_size} "
  "--add_dummy_prefix=false " #Add dummy whitespace at the beginning of sentence, default true
  "--character_coverage=1.0 "\
  # "--model_type=unigram "\
  "--model_type=bpe "\
  "--normalization_rule_name=identity "\
  "pad_id=0 "\
  "unk_id=1 "\
  "bos_id=2 "\
  "eos_id=3 "\
  "pad_piece=[PAD] "\
  "unk_piece=[UNK] "\
  "bos_piece=[CLS] "\
  "eos_piece=[SEP] "\
)



In [23]:
sp = spm.SentencePieceProcessor(model_file='myBPE.model')
enc = sp.Encode('ACTTGGTTAAACCTTTTCAAAAAAAAAAAAAAAAAAA')
print([sp.IdToPiece(id) for id in enc])


['ACTTGGTT', 'AAACCTTTT', 'CAA', 'AAAAAAAAAAAAAAAA', 'A']


In [24]:
vocabs = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]
print(vocabs[:10])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', 'AT', 'AG', 'CT', 'AA', 'TG', 'AC']


In [25]:
sorted_vocab = sorted(vocabs,key=len)
print(sorted_vocab[:10])
print(sorted_vocab[-10:])
print(len(sorted_vocab[-1]))

['T', 'A', 'G', 'C', 'N', 'AT', 'AG', 'CT', 'AA', 'TG']
['AGGTTGCCTGTTCACTCTGATGGTAGTTTCTTTTGCTGTGCAGAAGCTCTTTAGTTTAATTAGATCCCATTTGTCAATTT', 'AAAAAGTCAGGAAACAACAGGTGCTGGAGAGGATGTGGAGAAATAGGAACACTTTTACACTGTTGGTGGGACTGTAAACTAGTT', 'CCCTTTAGACAGAGCAGATTTGAAACACTCTTTTTGCGGAATTTGCAAGTGGAGATTTCTAGCCATTTGATGCCAACAGTAGAAAGGG', 'AAGAAGAGTTCTCAGTAACTTTTTTGTGTTGTGTGTATTCAACTCACAGAGTTGAACCTTGCTTTAGAGAGAGCAGATTTGAAACACT', 'CGATTGGAGATTTGGACTGCTTTGAGGCCTACTGTAGTAAAGGAAATAACTTCATCTAAAAACCAAACGGAAGCATTCACAGACAATTCTT', 'CTAGTTTTGAGGATTTCGTTGGAAGCGGGAATTCATACAAATTGCAGACTGCAGCGTTCTGAGAAACATCTTTGTGATGTTTGTATTCAGG', 'ACGACATAGAAGCATTCTCAGAAACTGCTCTGTGATGATTGCATTCAACTCCCAGAGTTGAACATTCCTTTTGATAGAGCAGTTTGCAAACACT', 'CCCACCAACAGTGTAAAAGTGTTCCTATTTCTCCACATCCTCTCCAGCACCTGTTGTTTCCTGACTTTTTAATGATTGCCATTCTAACTGGTGTGAG', 'CTTTTCTTAGAGCAGTTTAGAAACACTCTGCTTGTTATGTCTGCAAGTGGATATTTGGACCTCTTTGAGGCCTTCGTTGCAAACGGGGTTTCTTCCTTT', 'CCTTCCTTCGAAACGGGTATATCTTCACATCAAACCTAGACAGAAGCATTCTCAGAATGTTTCCTGTGATGACTGCATTCAACTCACAGAGGTG

In [26]:
!wget https://raw.githubusercontent.com/huggingface/tokenizers/main/bindings/python/scripts/sentencepiece_extractor.py

--2022-07-22 22:34:54--  https://raw.githubusercontent.com/huggingface/tokenizers/main/bindings/python/scripts/sentencepiece_extractor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4891 (4.8K) [text/plain]
Saving to: ‘sentencepiece_extractor.py.2’

sentencepiece_extra 100%[===================>]   4.78K  --.-KB/s    in 0s      

2022-07-22 22:34:54 (30.3 MB/s) - ‘sentencepiece_extractor.py.2’ saved [4891/4891]



In [27]:
from sentencepiece_extractor import SentencePieceExtractor

extractor = SentencePieceExtractor('myBPE.model')
vocab, merges = extractor.extract()

100%|██████████| 29999/29999 [04:12<00:00, 118.93it/s]


In [28]:
tokenizer = SentencePieceBPETokenizer(vocab=vocab, merges=merges, add_prefix_space=False, fuse_unk=False, unk_token='[UNK]')
tokenizer.add_special_tokens(["[MASK]"])
tokenizer_name = f'DNA_Sentencepiece_vocab_{tokenizer.get_vocab_size()}'
print('size', tokenizer.get_vocab_size())
print('[MASK]', tokenizer.token_to_id('[MASK]'))
print('[PAD]', tokenizer.token_to_id('[PAD]'))
print('[BOS]', tokenizer.token_to_id('[BOS]'))
print('[EOS]', tokenizer.token_to_id('[EOS]'))
print('[UNK]', tokenizer.token_to_id('[UNK]'))
print('[SEP]', tokenizer.token_to_id('[SEP]'))
print('[CLS]', tokenizer.token_to_id('[CLS]'))

tokenizer.post_processor = BertProcessing(sep=('[SEP]', tokenizer.token_to_id('[SEP]')), cls=('[CLS]', tokenizer.token_to_id('[CLS]')))




size 30000
[MASK] 29999
[PAD] 0
[BOS] None
[EOS] None
[UNK] 1
[SEP] 3
[CLS] 2


# Validation

In [29]:
tokens = sorted(tokenizer.get_vocab().keys(), key=len)
print('SHORTEST TOKENS\n', *tokens[:10])
print(f'\nLONGEST TOKENS (max {len(tokens[-1])})', *tokens[-10:], sep='\n')
tokenizer_name = tokenizer_name+f'_max_tokenlen_{len(tokens[-1])}'

SHORTEST TOKENS
 C N A G T AC GG TG GT AT

LONGEST TOKENS (max 100)
AGGTTGCCTGTTCACTCTGATGGTAGTTTCTTTTGCTGTGCAGAAGCTCTTTAGTTTAATTAGATCCCATTTGTCAATTT
AAAAAGTCAGGAAACAACAGGTGCTGGAGAGGATGTGGAGAAATAGGAACACTTTTACACTGTTGGTGGGACTGTAAACTAGTT
AAGAAGAGTTCTCAGTAACTTTTTTGTGTTGTGTGTATTCAACTCACAGAGTTGAACCTTGCTTTAGAGAGAGCAGATTTGAAACACT
CCCTTTAGACAGAGCAGATTTGAAACACTCTTTTTGCGGAATTTGCAAGTGGAGATTTCTAGCCATTTGATGCCAACAGTAGAAAGGG
CGATTGGAGATTTGGACTGCTTTGAGGCCTACTGTAGTAAAGGAAATAACTTCATCTAAAAACCAAACGGAAGCATTCACAGACAATTCTT
CTAGTTTTGAGGATTTCGTTGGAAGCGGGAATTCATACAAATTGCAGACTGCAGCGTTCTGAGAAACATCTTTGTGATGTTTGTATTCAGG
ACGACATAGAAGCATTCTCAGAAACTGCTCTGTGATGATTGCATTCAACTCCCAGAGTTGAACATTCCTTTTGATAGAGCAGTTTGCAAACACT
CCCACCAACAGTGTAAAAGTGTTCCTATTTCTCCACATCCTCTCCAGCACCTGTTGTTTCCTGACTTTTTAATGATTGCCATTCTAACTGGTGTGAG
CTTTTCTTAGAGCAGTTTAGAAACACTCTGCTTGTTATGTCTGCAAGTGGATATTTGGACCTCTTTGAGGCCTTCGTTGCAAACGGGGTTTCTTCCTTT
CCTTCCTTCGAAACGGGTATATCTTCACATCAAACCTAGACAGAAGCATTCTCAGAATGTTTCCTGTGATGACTGCATTCAACTCACAGAGGTGAACAAT


In [30]:
encoding = tokenizer.encode('NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG[MASK]')
print(encoding.ids)
print(encoding.tokens)
print(tokenizer.decode(encoding.ids))

[2, 29998, 29998, 40, 639, 32, 442, 29997, 29999, 29996, 9685, 461, 29999, 3]
['[CLS]', 'N', 'N', 'ACTG', 'ACACG', 'AAAA', 'AAAGGG', 'C', '[MASK]', 'G', 'CGCAA', 'CTCCAG', '[MASK]', '[SEP]']
[CLS]NNACTGACACGAAAAAAAGGGCGCGCAACTCCAG[SEP]


In [31]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    model_max_length = 512,
    padding_side='right',
    truncation_side='right',
    pad_token='[PAD]',
    mask_token='[MASK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    unk_token='[UNK]',
  )

fast_tokenizer.push_to_hub(tokenizer_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='DNA_Sentencepiece_vocab_30000_max_tokenlen_100' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Vlasta/DNA_Sentencepiece_vocab_30000_max_tokenlen_100 into local empty directory.
To https://huggingface.co/Vlasta/DNA_Sentencepiece_vocab_30000_max_tokenlen_100
   c06ff80..2259c27  main -> main

   c06ff80..2259c27  main -> main



'https://huggingface.co/Vlasta/DNA_Sentencepiece_vocab_30000_max_tokenlen_100/commit/2259c278b1a8231f079fac547480bcbfcafe4694'

In [32]:
e = fast_tokenizer('NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG')
d = fast_tokenizer.decode(e['input_ids'])
print(e)
print(d)

{'input_ids': [2, 29998, 29998, 40, 639, 32, 442, 29997, 29999, 29996, 9685, 461, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS]NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG[SEP]


In [34]:
from transformers import AutoTokenizer
myDownloadedTokenizer = AutoTokenizer.from_pretrained(f"{hf_name}/{tokenizer_name}")
# myDownloadedTokenizer = fast_tokenizer
dnabert_tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

long_sequence = '[MASK]' + 'AAAAAA '*1000 
short_sequence = 'NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG'
long_encoded_dnabert = dnabert_tokenizer(long_sequence)
long_encoded_ours = myDownloadedTokenizer(long_sequence)
#Padding doesnt show up for
short_encoded_dnabert = dnabert_tokenizer(short_sequence, padding=True)
short_encoded_ours = myDownloadedTokenizer(short_sequence, padding=True)
print("\nDNABERT BEHAVIOR")
print(long_encoded_dnabert)
print(dnabert_tokenizer.decode(long_encoded_dnabert['input_ids']))
print(short_encoded_dnabert)
print(dnabert_tokenizer.decode(short_encoded_dnabert['input_ids']))
print(dnabert_tokenizer.special_tokens_map)
print(dnabert_tokenizer.padding_side)
print('\nOURS BEHAVIOR')
print(long_encoded_ours)
print(myDownloadedTokenizer.decode(long_encoded_ours['input_ids']))
print(short_encoded_ours)
print(myDownloadedTokenizer.decode(short_encoded_ours['input_ids']))
print(myDownloadedTokenizer.special_tokens_map)
print(myDownloadedTokenizer.padding_side)





Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1003 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3003 > 512). Running this sequence through the model will result in indexing errors



DNABERT BEHAVIOR
{'input_ids': [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5